In [36]:
import pulp
import pandas as pd
from pulp import LpMaximize, LpProblem, LpVariable, LpBinary ,LpMinimize,value
import random
from datetime import datetime

In [40]:
df = pd.read_csv("../data/normalized_shift_preferences.csv")
staff_df = pd.read_csv("../data/__________20_____.csv")
staff_df.head()

,ID,Name,Level,Gender,Age,Email,status
0,1001,khh,3,Male,24,khh@gmail.com,留学生
1,1002,kph,3,Female,24,kph,留学生
2,1003,yss,3,Male,24,yss@gmail.com,International Student
3,1004,user1004,1,Female,28,user1004@example.com,Regular
4,1005,user1005,5,Female,28,user1005@example.com,留学生


In [38]:
df.head()


,id,staff_id,date,shift_type,preferred,staff_level
0,0,1001,2025-07-01,morning,True,3
1,1,1001,2025-07-01,afternoon,True,3
2,2,1001,2025-07-01,night,False,3
3,3,1001,2025-07-02,morning,False,3
4,4,1001,2025-07-02,afternoon,True,3


In [39]:
from pulp import LpProblem, LpVariable, lpSum, LpMinimize, PULP_CBC_CMD
import pandas as pd

# 時間帯マップ
time_map = {
    "morning": list(range(9, 13)),      # 4時間
    "afternoon": list(range(13, 18)),   # 5時間
    "night": list(range(18, 25))        # 6時間
}

# 必要人数ルール
def get_required_staff(hour):
    if 9 <= hour <= 11:
        return 2
    elif 15 <= hour <= 17 or 20 <= hour <= 24:
        return 3
    else:
        return 4

# スタッフが働けるかチェック
def can_work(staff_row, hour):
    if staff_row["age"] < 18 and hour >= 22:
        return False
    if staff_row["status"] == "highschool" and hour >= 22:
        return False
    return True

# ダミーデータ（希望）
"""
df = pd.DataFrame([
    {"staff_id": 1, "date": "2025-07-21", "shift_type": "morning", "preferred": 1},
    {"staff_id": 1, "date": "2025-07-21", "shift_type": "afternoon", "preferred": 1},
    {"staff_id": 1, "date": "2025-07-22", "shift_type": "night", "preferred": 1},
    {"staff_id": 2, "date": "2025-07-21", "shift_type": "afternoon", "preferred": 1},
    {"staff_id": 3, "date": "2025-07-21", "shift_type": "night", "preferred": 1},
])

# スタッフ情報
staff_df = pd.DataFrame([
    {"staff_id": 1, "age": 20, "status": "international"},
    {"staff_id": 2, "age": 17, "status": "highschool"},
    {"staff_id": 3, "age": 25, "status": "fulltime"},
])
"""
# モデル定義
model = LpProblem("Shift_Assignment", LpMinimize)

# 変数定義
x_var = {}
for _, row in df.iterrows():
    if row["preferred"] == 1:
        key = f"x_{row.staff_id}_{row.date}_{row.shift_type}"
        x_var[key] = LpVariable(key, cat="Binary")

# 人数制約
for date in df["date"].unique():
    for hour in range(9, 25):
        required = get_required_staff(hour)
        valid_vars = []
        for _, row in df[(df["date"] == date) & (df["preferred"] == 1)].iterrows():
            staff_info = staff_df[staff_df["ID"] == row["ID"]].iloc[0]
            if can_work(staff_info, hour):
                if hour in time_map.get(row["shift_type"], []):
                    key = f"x_{row.staff_id}_{row.date}_{row.shift_type}"
                    if key in x_var:
                        valid_vars.append(x_var[key])
        model += lpSum(valid_vars) >= required

# ✅ 留学生：週28時間以内（各シフト時間を加算）
for staff_id in df["staff_id"].unique():
    staff_info = staff_df[staff_df["ID"] == staff_id].iloc[0]
    if staff_info["status"] == "International Student" or staff_info["status"] == "留学生":
        terms = []
        for key in x_var:
            _, sid, _, shift_type = key.split("_")
            if int(sid) == staff_id:
                hours = len(time_map[shift_type])
                terms.append(x_var[key] * hours)
        model += lpSum(terms) <= 28

# ✅ 1日6時間以上→休憩1h → 2シフトまでOKに制限
for staff_id in df["staff_id"].unique():
    for date in df["date"].unique():
        shift_count = []
        for shift_type in time_map:
            key = f"x_{staff_id}_{date}_{shift_type}"
            if key in x_var:
                shift_count.append(x_var[key])
        model += lpSum(shift_count) <= 2  # 2シフトまで＝最大でも11時間程度（1h休憩込み）

# 目的関数（希望最大化）
model += lpSum(x_var.values())

# 解を求める
model.solve(PULP_CBC_CMD(msg=0))

# 結果表示
for key, var in x_var.items():
    if var.value() == 1:
        print(key, "=> assigned")


KeyError: 'ID'

In [42]:
from pulp import LpProblem, LpVariable, lpSum, LpMinimize, PULP_CBC_CMD
import pandas as pd

# 時間帯マップ
time_map = {
    "morning": list(range(9, 13)),
    "afternoon": list(range(13, 18)),
    "night": list(range(18, 25))
}

def get_required_staff(hour):
    if 9 <= hour <= 11:
        return 2
    elif 15 <= hour <= 17 or 20 <= hour <= 24:
        return 3
    else:
        return 4

def can_work(staff_row, hour):
    if staff_row["Age"] < 18 and hour >= 22:
        return False
    if staff_row["status"] == "highschool" and hour >= 22:
        return False
    return True

# CSV読み込み（あなたのスクショに合わせて修正）
df = pd.read_csv("../data/normalized_shift_preferences.csv")
staff_df = pd.read_csv("../data/__________20_____.csv")  # 正しいファイル名に差し替えてください

# モデル定義
model = LpProblem("Shift_Assignment", LpMinimize)

# 変数定義
x_var = {}
for _, row in df.iterrows():
    if row["preferred"] == True:
        key = f"x_{row.staff_id}_{row.date}_{row.shift_type}"
        x_var[key] = LpVariable(key, cat="Binary")

# 人数制約
for date in df["date"].unique():
    for hour in range(9, 25):
        required = get_required_staff(hour)
        valid_vars = []
        for _, row in df[(df["date"] == date) & (df["preferred"] == True)].iterrows():
            staff_info = staff_df[staff_df["ID"] == row["staff_id"]].iloc[0]
            if can_work(staff_info, hour):
                if hour in time_map.get(row["shift_type"], []):
                    key = f"x_{row.staff_id}_{row.date}_{row.shift_type}"
                    if key in x_var:
                        valid_vars.append(x_var[key])
        model += lpSum(valid_vars) >= required

# 留学生週28時間制限
for staff_id in df["staff_id"].unique():
    staff_info = staff_df[staff_df["ID"] == staff_id].iloc[0]
    if staff_info["status"] in ["International Student", "留学生"]:
        terms = []
        for key in x_var:
            _, sid, _, shift_type = key.split("_")
            if int(sid) == staff_id:
                hours = len(time_map[shift_type])
                terms.append(x_var[key] * hours)
        model += lpSum(terms) <= 28

# 1日2シフトまで（休憩考慮）
for staff_id in df["staff_id"].unique():
    for date in df["date"].unique():
        shift_count = []
        for shift_type in time_map:
            key = f"x_{staff_id}_{date}_{shift_type}"
            if key in x_var:
                shift_count.append(x_var[key])
        model += lpSum(shift_count) <= 2



# ✅ レベル制約の追加（各時間帯でレベル合計が15以上必要）
for date in df["date"].unique():
    for hour in range(9, 25):
        level_vars = []
        for _, row in df[(df["date"] == date) & (df["preferred"] == True)].iterrows():
            staff_info = staff_df[staff_df["ID"] == row["staff_id"]].iloc[0]
            if can_work(staff_info, hour):
                if hour in time_map.get(row["shift_type"], []):
                    key = f"x_{row.staff_id}_{row.date}_{row.shift_type}"
                    if key in x_var:
                        # スタッフのレベル × その時間に働くかどうか（バイナリ変数）
                        level_vars.append(x_var[key] * row["staff_level"])
        model += lpSum(level_vars) >= 15  # レベル合計が15以上必要

# 目的関数（希望を最大化）
model += lpSum(x_var.values())

# 解く
model.solve(PULP_CBC_CMD(msg=0))

# 結果表示
for key, var in x_var.items():
    if var.value() == 1:
        print(key, "=> assigned")


x_1001_2025-07-01_morning => assigned
x_1001_2025-07-02_night => assigned
x_1001_2025-07-03_afternoon => assigned
x_1001_2025-07-04_afternoon => assigned
x_1001_2025-07-05_night => assigned
x_1002_2025-07-03_afternoon => assigned
x_1002_2025-07-04_morning => assigned
x_1002_2025-07-05_afternoon => assigned
x_1002_2025-07-06_night => assigned
x_1003_2025-07-01_night => assigned
x_1003_2025-07-02_morning => assigned
x_1003_2025-07-04_morning => assigned
x_1003_2025-07-06_afternoon => assigned
x_1003_2025-07-07_morning => assigned
x_1004_2025-07-03_morning => assigned
x_1005_2025-07-01_morning => assigned
x_1005_2025-07-03_morning => assigned
x_1005_2025-07-03_night => assigned
x_1005_2025-07-04_afternoon => assigned
x_1005_2025-07-06_morning => assigned
x_1006_2025-07-03_afternoon => assigned
x_1006_2025-07-05_morning => assigned
x_1007_2025-07-05_morning => assigned
x_1007_2025-07-07_night => assigned
x_1008_2025-07-01_afternoon => assigned
x_1008_2025-07-01_night => assigned
x_1008_202